In [6]:
import  cv2
import numpy as np
import os
from os import listdir
from os.path import isfile, join
import boto3

In [7]:
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [8]:
def face_extractor(img):
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=face_classifier.detectMultiScale(gray,1.3,5)
    
    if faces is ():
        return None
    for (x,y,w,h) in faces:
        cropped_face = img[y:y+h, x:x+w]
        return cropped_face
cap=cv2.VideoCapture(0)
count=0
while True:
    ret, frame = cap.read()
    if face_extractor(frame) is not None:
        count += 1
        face = cv2.resize(face_extractor(frame), (200, 200))
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        file_name_path = 'C:/Users/priya/Desktop/Mlops/Images/' + str(count) + '.jpg'
        cv2.imwrite(file_name_path, face)

        cv2.putText(face, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        cv2.imshow('Face Cropper', face)
        
    else:
        print("Face not found")
        pass

    if cv2.waitKey(1) == 13 or count == 100: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()      
print("Collecting Samples Complete")

<>:5: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:5: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-8-11cb74392e50>:5: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():


Face not found


In [9]:
# Get the training data we previously made
data_path = 'C:/Users/priya/Desktop/Mlops/Images/'
onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path, f))]

# Create arrays for training data and labels
Training_Data, Labels = [], []

# Open training images in our datapath
# Create a numpy array for training data
for i, files in enumerate(onlyfiles):
    image_path = data_path + onlyfiles[i]
    images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    Training_Data.append(np.asarray(images, dtype=np.uint8))
    Labels.append(i)

# Create a numpy array for both training data and labels
Labels = np.asarray(Labels, dtype=np.int32)

# Initialize facial recognizer
# model = cv2.face.createLBPHFaceRecognizer()
# NOTE: For OpenCV 3.0 use cv2.face.createLBPHFaceRecognizer()
# pip install opencv-contrib-python
# model = cv2.createLBPHFaceRecognizer()

model  = cv2.face_LBPHFaceRecognizer.create()
# Let's train our model 
model.train(np.asarray(Training_Data), np.asarray(Labels))
print("Model trained sucessefully")

Model trained sucessefully


In [10]:
import cv2
import numpy as np
import os


face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

def face_detector(img, size=0.5):
    
    # Convert image to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is ():
        return img, []
    
    
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        roi = img[y:y+h, x:x+w]
        roi = cv2.resize(roi, (200, 200))
    return img, roi


# Open Webcam
cap = cv2.VideoCapture(0)

while True:

    ret, frame = cap.read()
    
    image, face = face_detector(frame)
    
    try:
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        # Pass face to prediction model
        # "results" comprises of a tuple containing the label and the confidence value
        results = model.predict(face)
        # harry_model.predict(face)
        
        if results[1] < 500:
            confidence = int( 100 * (1 - (results[1])/400) )
            display_string = str(confidence) + '% Confident it is User'
            
        cv2.putText(image, display_string, (100, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (255,120,150), 2)
        
        if confidence > 85:
            cv2.putText(image, "Hey Kontham ", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            cv2.imshow('Face Recognition', image )
#             
            
        
               #task6.3(Create EC2 instance in the AWS using CLI.  )
            import boto3
            import os
            print("creating EC2 Instance....")
            resource_ec2=boto3.client("ec2")
            ec2_id = resource_ec2.run_instances(
            ImageId="ami-0ad704c126371a549",
            MinCount=1,
            MaxCount=1,
            InstanceType="t2.micro",
            KeyName="MLOps AWS key"
            )
            print("instance Created successfully...")
            #task6.4create and attach ebs volume to ec2 instance
            #creating EBS volume 
            print("Creating EBS volume of 10 GB...")
            client = boto3.client('ec2')
            volume_id = client.create_volume( AvailabilityZone='ap-south-1a',
            Size=10,
            VolumeType='gp2',
            TagSpecifications=[
                {
                    'ResourceType': 'volume',
                    'Tags': [
                        {
                            'Key': 'Name',
                            'Value': 'task6'
                        },
                            ]
                },
            ],)
            instanceid='ec2_id["Instances"][0]["InstanceId"]'
            vid = 'volume_id["VolumeId"]'
            print("Volume  created successfully...\n")
            print(" Attaching EBS volume to EC2 instance...")
#ec2.client.attach_volume(device='/dev/xvdb',
#            InsatanceId=instanceid,
#            VolumeId = vid            
#            ) 
            os.system("aws ec2 attach-volume --volume-id vid --instance-id instanceid --device /dev/xvdb")
            print("volume attached successfully...")

        
        
         
        else:
            
            cv2.putText(image, "I dont know, how r u", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
            cv2.imshow('Face Recognition', image )

    except:
        cv2.putText(image, "No Face Found", (220, 120) , cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.putText(image, "looking for face", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.imshow('Face Recognition', image )
        pass
        
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()    

<>:13: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:13: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-10-6649b0b6cca9>:13: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():


creating EC2 Instance....
instance Created successfully...
Creating EBS volume of 10 GB...
Volume  created successfully...

 Attaching EBS volume to EC2 instance...
volume attached successfully...
creating EC2 Instance....
instance Created successfully...
Creating EBS volume of 10 GB...
Volume  created successfully...

 Attaching EBS volume to EC2 instance...
volume attached successfully...


In [ ]:
    #task6.3(Create EC2 instance in the AWS using CLI.  )
import boto3
import os
print("creating EC2 Instance....")
resource_ec2=boto3.client("ec2")
ec2_id = resource_ec2.run_instances(
ImageId="ami-0ad704c126371a549",
MinCount=1,
MaxCount=1,
InstanceType="t2.micro",
KeyName="MLOps AWS key"
)
print("instance Created successfully...")
            #task6.4create and attach ebs volume to ec2 instance
            #creating EBS volume 
print("Creating EBS volume of 10 GB...")
client = boto3.client('ec2')
volume_id = client.create_volume( AvailabilityZone='ap-south-1a',
Size=10,
VolumeType='gp2',
TagSpecifications=[
    {
        'ResourceType': 'volume',
        'Tags': [
            {
                'Key': 'Name',
                'Value': 'task6'
            },
                ]
    },
],)
instanceid='ec2_id["Instances"][0]["InstanceId"]'
vid = 'volume_id["VolumeId"]'
print("Volume  created successfully...\n")
print(" Attaching EBS volume to EC2 instance...")
#ec2.client.attach_volume(device='/dev/xvdb',
#            InsatanceId=instanceid,
#            VolumeId = vid            
#            ) 
os.system("aws ec2 attach-volume --volume-id vid --instance-id instanceid --device /dev/xvdb")
print("volume attached successfully...")
